In [1]:
# 1) Install dependencies
!pip install -q gradio==3.44.0 transformers==4.33.2 torch torchvision sentence-transformers faiss-cpu gTTS git+https://github.com/openai/whisper.git sqlite-utils

# If some packages warn about versions, ignore for prototype.
print("Dependencies installed")


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)
Dependencies installed


In [2]:
!pip install whisper gtts

  Using cached whisper-1.1.10-py3-none-any.whl


In [3]:
!pip install faiss-cpu

In [4]:
!pip uninstall -y whisper


Found existing installation: whisper 1.1.10
Uninstalling whisper-1.1.10:
  Successfully uninstalled whisper-1.1.10


In [5]:
!pip install openai-whisper


In [6]:
# Colab prototype: Multi-Intent AI Farmer Assistant (Malayalam-first)
# --------------------------------------------------------------------------------
# Run these cells in order. You'll need to provide an OpenWeather API key for weather.
# Replace placeholders (image model, market API) with production components later.
# --------------------------------------------------------------------------------



# ---------------------------
# 2) Imports & basic setup
# ---------------------------
import os
import json
import random
import tempfile
import sqlite3
import time
from typing import Tuple

import gradio as gr
import torch
import whisper
from gtts import gTTS
from PIL import Image

from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ---------------------------
# 3) Config - put keys / settings here
# ---------------------------
OPENWEATHER_API_KEY = "YOUR_OPENWEATHER_API_KEY_HERE"  # <- Put your OpenWeather API key here
DEFAULT_CITY = "Thrissur,IN"  # default fallback location (City,CountryCode)

# Escalation confidence threshold: if below, create escalation
ESCALATION_THRESHOLD = 0.45

# ---------------------------
# 4) Lightweight intent classifier (rule-based + small heuristics)
#    You can replace with ML classifier later.
# ---------------------------
INTENTS = ["weather", "pest", "scheme", "market", "advice"]

# simple keywords in Malayalam/English for heuristic routing
INTENT_KEYWORDS = {
    "weather": {"വേന"}, # keep empty for fallback - will use english too
}

# I'll use a more practical keyword map (Malayalam + English)
INTENT_KEYWORDS = {
    "weather": ["മഴ", "വെയില്", "കാലാവസ്ഥ", "rain", "weather", "തിങ്കളാഴ്ച", "ഇന്ന്", "നാളെ"],
    "pest": ["രോഗം", "പോക്കുകള്", "leaf", "leaf spot", "പൊപ്പ്", "pest", "പീസ്റ്റ്", "ഇല", "പൂണ്ട"],
    "scheme": ["പദ്ധതി", "PM-Kisan", "പദ്ധതി", "ഷീം", "സബ്സിഡി", "സഹായം", "രൂപ", "പദ്ധതികള്"],
    "market": ["വില", "market", "കർഷക বজാർ", "മാർക്കറ്റ്", "മാങ്ങ വില", "കണ"],
    "advice": ["എങ്ങനെ", "how to", "തടവുകൾ", "കൂടാതെ", "അടക്കം", "എന്ത് ചെയ്യാം"]
}

def detect_intent(text: str) -> str:
    txt = text.lower()
    # simple keyword matching
    for intent, kws in INTENT_KEYWORDS.items():
        for kw in kws:
            if kw.lower() in txt:
                return intent
    # fallback rules
    if "price" in txt or "വില" in txt:
        return "market"
    if "scheme" in txt or "പദ്ധതി" in txt or "pm-kisan" in txt.lower():
        return "scheme"
    # default to advice
    return "advice"

# ---------------------------
# 5) Setup Whisper (Malayalam ASR)
# ---------------------------
print("Loading Whisper ASR (small). This may take ~30s - 2 min depending on runtime.")
asr_model = whisper.load_model("small")
def transcribe_audio(filepath):
    if filepath is None:
        return ""
    # Whisper expects a path
    try:
        result = asr_model.transcribe(filepath, language="ml", fp16=False)
        return result.get("text", "").strip()
    except Exception as e:
        print("ASR error:", e)
        return ""

# ---------------------------
# 6) Setup RAG embedding + simple KB with FAISS
# ---------------------------
print("Loading sentence-transformers embedder...")
embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# A small Kerala-focused KB (extend this with official docs & advisories)
documents = [
    {
        "id": "kb1",
        "title": "Banana leaf spot advice",
        "text_ml": "ബനാനയിലെ leaf spot രോഗത്തിന് Mancozeb 0.2% സ്പ്രേ ചെയ്തു, 10-14 ദിവസം ഇടവേളയിൽ രണ്ട് തവണ ആവർത്തിക്കുക. സുരക്ഷ മുൻനിർത്തി പിപിഇ ധരിക്കുക.",
        "text_en": "For banana leaf spot, use Mancozeb 0.2% spray, repeat after 10-14 days. Use PPE."
    },
    {
        "id": "kb2",
        "title": "Paddy fertilizer September",
        "text_ml": "നെല്ലിന് September-ൽ NPK 20:10:10 ഉപയോഗിക്കാൻ ശിപാർശ ചെയ്യുന്നു. നാഴികമൂല്യങ്ങൾ സ്ഥലം പ്രകടം ചെയ്‌ത് ചിന്തിക്കുക.",
        "text_en": "For paddy in September, recommended NPK 20:10:10."
    },
    {
        "id": "kb3",
        "title": "PM-Kisan scheme (India)",
        "text_ml": "PM-Kisan പദ്ധതിയിൽ അർഹതയുള്ള കർഷകർക്ക് കേന്ദ്ര حکومتത്തിൽ നിന്നുള്ള സാമ്പത്തിക സഹായം. അപേക്ഷിക്കാനുള്ള ലിങ്ക്: https://pmkisan.gov.in",
        "text_en": "PM-Kisan provides direct income support to eligible farmers in India."
    }
]

doc_texts = [d["text_ml"] + " " + d["text_en"] for d in documents]  # multilingual text
doc_embeddings = embedder.encode(doc_texts, convert_to_tensor=False, show_progress_bar=False)

import numpy as np
doc_embeddings = np.vstack(doc_embeddings).astype("float32")

# Build FAISS index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

def retrieve_kb(query: str, k: int = 2):
    q_emb = embedder.encode([query], convert_to_tensor=False)
    q_emb = np.array(q_emb).astype("float32")
    D, I = index.search(q_emb, k)
    results = []
    for idx in I[0]:
        results.append(doc_texts[idx])
    return results

# ---------------------------
# 7) LLM for answer generation (small seq2seq model used as demo)
# ---------------------------
print("Loading a small seq2seq model for generation... (demo)")
llm_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = AutoTokenizer.from_pretrained(llm_name)
llm = AutoModelForSeq2SeqLM.from_pretrained(llm_name)

def generate_answer(query: str, context_docs: list, max_length: int = 150) -> Tuple[str, float]:
    # Create input by concatenating query + contexts
    input_text = "Query: " + query + " Context: " + " || ".join(context_docs)
    inputs = tokenizer([input_text], return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = llm.generate(**inputs, max_length=max_length, num_beams=3, early_stopping=True)
    ans = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Simple confidence heuristic: longer overlap with KB -> higher confidence
    confidence = min(0.95, 0.3 + 0.1 * len(context_docs))
    return ans, confidence

# ---------------------------
# 8) Weather API utility (OpenWeather) - needs API key
# ---------------------------
import requests

def get_weather_for_city(city_query: str):
    """
    city_query: "Thrissur,IN" or "Kochi,IN" or "Lat, Lon"
    """
    if OPENWEATHER_API_KEY == "YOUR_OPENWEATHER_API_KEY_HERE":
        return "OpenWeather API key not set. Please set OPENWEATHER_API_KEY in the notebook."

    # Using current weather + short forecast
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city_query,
        "appid": OPENWEATHER_API_KEY,
        "units": "metric"
    }
    try:
        r = requests.get(base_url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()
        weather_desc = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        wind = data["wind"]["speed"]
        return f"Location: {city_query}. Weather: {weather_desc}. Temp: {temp}°C. Humidity: {humidity}%. Wind: {wind} m/s."
    except Exception as e:
        return f"Weather API error: {e}"

# ---------------------------
# 9) Market price mock (replace with Agmarknet or district API)
# ---------------------------
# Simple mocked market prices for demo; you should replace with real API integration.
MOCK_MARKET = {
    "coconut": {"Kochi": "₹30/kg", "Thrissur": "₹28/kg"},
    "banana": {"Kochi": "₹20/kg", "Thrissur": "₹18/kg"},
    "paddy": {"Kochi": "₹1800/quintal", "Thrissur": "₹1750/quintal"}
}

def get_market_price(item: str, district: str = "Thrissur"):
    itm = item.lower().strip()
    if itm in MOCK_MARKET:
        price = MOCK_MARKET[itm].get(district, "Price not available for district")
        return f"Market price for {item} in {district}: {price}"
    else:
        return f"No market data for {item} in demo. Replace with Agmarknet API."

# ---------------------------
# 10) Simple image disease classifier placeholder
# ---------------------------
DISEASE_CLASSES = ["Healthy", "Leaf Spot", "Rust", "Blight", "Unknown"]

def classify_image_placeholder(image_path: str):
    # This is a placeholder - use a trained model for real predictions
    # For the demo, randomly choose a label but return helpful wording.
    label = random.choice(DISEASE_CLASSES)
    advice = {
        "Healthy": "Leaf appears healthy. Monitor crop and follow routine care.",
        "Leaf Spot": "Possible Leaf Spot. Consider copper/mancozeb spray as per label. Send more photos or escalate if unsure.",
        "Rust": "Symptoms look like Rust. Remove affected leaves and consult extension officer.",
        "Blight": "Possible Blight. Escalate to expert for confirmation.",
        "Unknown": "Unable to classify. Please provide clearer images and farmer history."
    }
    return label, advice[label]

# ---------------------------
# 11) SQLite storage for history & escalations
# ---------------------------
conn = sqlite3.connect("farmer_assistant.db", check_same_thread=False)
c = conn.cursor()
c.execute("""CREATE TABLE IF NOT EXISTS queries (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                timestamp REAL,
                user_phone TEXT,
                intent TEXT,
                query TEXT,
                response TEXT,
                confidence REAL,
                escalated INTEGER
            )""")
c.execute("""CREATE TABLE IF NOT EXISTS escalations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                query_id INTEGER,
                timestamp REAL,
                officer_notes TEXT,
                status TEXT
            )""")
conn.commit()

def save_query(user_phone, intent, query, response, confidence, escalated):
    ts = time.time()
    c.execute("INSERT INTO queries (timestamp, user_phone, intent, query, response, confidence, escalated) VALUES (?,?,?,?,?,?,?)",
              (ts, user_phone, intent, query, response, confidence, 1 if escalated else 0))
    conn.commit()
    return c.lastrowid

def create_escalation(query_id, note="Auto-escalation"):
    ts = time.time()
    c.execute("INSERT INTO escalations (query_id, timestamp, officer_notes, status) VALUES (?,?,?,?)",
              (query_id, ts, note, "open"))
    conn.commit()
    return c.lastrowid

# ---------------------------
# 12) TTS wrapper (Malayalam)
# ---------------------------
def tts_malayalam(text: str):
    try:
        tts = gTTS(text=text, lang="ml")
        fn = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        tts.save(fn.name)
        return fn.name
    except Exception as e:
        print("TTS error:", e)
        return None

# ---------------------------
# 13) Main orchestration function
# ---------------------------
def handle_query(user_phone: str, text_input: str, image_path: str = None, city_hint: str = DEFAULT_CITY):
    """
    :param user_phone: phone or identifier (for demo)
    :param text_input: farmer's text in Malayalam or English
    :param image_path: optional uploaded image path
    :param city_hint: used for weather/market queries
    """
    # 1) Intent detection
    intent = detect_intent(text_input)

    # 2) If image present and intent seems pest/disease, run classifier
    image_label = None
    image_advice = None
    if image_path:
        image_label, image_advice = classify_image_placeholder(image_path)

    # 3) Route to appropriate engine
    if intent == "weather":
        # try to extract city/district from query (very naive)
        city = city_hint
        weather_txt = get_weather_for_city(city)
        # generate simple answer
        ans = f"{weather_txt} \n(ഈ വിവരങ്ങള്‍ OpenWeather-ൽ നിന്നാണ് ലഭിക്കുന്നത്.)"
        confidence = 0.85 if "Weather API key not set" not in weather_txt else 0.25

    elif intent == "market":
        # try to find commodity in text
        # naive extraction: look for known mock items
        found = None
        for itm in MOCK_MARKET.keys():
            if itm in text_input.lower():
                found = itm
                break
        if found:
            ans = get_market_price(found, district=city_hint.split(",")[0])
            confidence = 0.8
        else:
            ans = "ദയവായി സാധനത്തിന്റെ പേര് വ്യക്തമാക്കുക (ഉദാ: coconut, banana, paddy)."
            confidence = 0.4

    elif intent == "scheme":
        # Use RAG to find scheme docs
        contexts = retrieve_kb(text_input, k=2)
        gen_ans, conf = generate_answer(text_input, contexts)
        ans = f"{gen_ans}\n\nSources: {', '.join([d[:120] for d in contexts])}"
        confidence = conf

    elif intent == "pest":
        # If there is an image, include image classifier result plus KB retrieval
        contexts = retrieve_kb(text_input, k=2)
        gen_ans, conf = generate_answer(text_input, contexts)
        if image_label:
            ans = f"Image classification: {image_label}. Advice: {image_advice}\nLLM reply: {gen_ans}\nSources: {', '.join([d[:120] for d in contexts])}"
            confidence = min(0.9, conf + 0.05)
        else:
            ans = f"{gen_ans}\nSources: {', '.join([d[:120] for d in contexts])}"
            confidence = conf

    else:  # advice / default
        contexts = retrieve_kb(text_input, k=2)
        gen_ans, conf = generate_answer(text_input, contexts)
        ans = f"{gen_ans}\nSources: {', '.join([d[:120] for d in contexts])}"
        confidence = conf

    # 4) Decide escalation
    escalated = False
    if confidence < ESCALATION_THRESHOLD:
        # save & create escalation
        qid = save_query(user_phone, intent, text_input, ans, confidence, escalated=False)
        esc_id = create_escalation(qid, note="Auto-escalation due to low confidence")
        escalated = True
    else:
        qid = save_query(user_phone, intent, text_input, ans, confidence, escalated=False)

    # 5) produce TTS
    audio_path = tts_malayalam(ans)  # may be None in some envs

    return {
        "intent": intent,
        "answer": ans,
        "confidence": float(confidence),
        "escalated": escalated,
        "audio_path": audio_path
    }

# ---------------------------
# 14) Gradio UI to test
# ---------------------------
def gradio_handle(user_phone, text_input, image_file, city_hint):
    if image_file is not None:
        img_path = image_file
    else:
        img_path = None
    out = handle_query(user_phone or "anonymous", text_input or "", img_path, city_hint or DEFAULT_CITY)
    # Return answer text and audio if available
    return out["answer"], out["confidence"], "Yes" if out["escalated"] else "No", out["intent"], out["audio_path"]

with gr.Blocks() as demo:
    gr.Markdown("## 🌾 Multi-Intent Farmer Assistant (Colab Prototype)\nSources: demo KB + OpenWeather (if API key provided).")
    with gr.Row():
        with gr.Column(scale=2):
            phone = gr.Textbox(label="Phone (for history)", placeholder="e.g. +919876543210")
            city = gr.Textbox(label="City Hint (e.g. Thrissur,IN)", value=DEFAULT_CITY)
            text_in = gr.Textbox(label="Your question (Malayalam or English)", lines=3, placeholder="എന്ത് സംശയമാണ്? | English")
            img = gr.Image(label="Attach image (optional)", type="filepath")
            submit = gr.Button("Ask")
        with gr.Column(scale=1):
            answer_out = gr.Textbox(label="AI Answer", lines=10)
            conf_out = gr.Number(label="Confidence")
            esc_out = gr.Textbox(label="Escalated?")
            intent_out = gr.Textbox(label="Detected Intent")
            audio_player = gr.Audio(label="Answer Audio (Malayalam)")

    submit.click(fn=gradio_handle, inputs=[phone, text_in, img, city], outputs=[answer_out, conf_out, esc_out, intent_out, audio_player])

demo.launch(share=True)


Loading Whisper ASR (small). This may take ~30s - 2 min depending on runtime.


100%|███████████████████████████████████████| 461M/461M [00:33<00:00, 14.4MiB/s]


Loading sentence-transformers embedder...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://712a08cb012c470165.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
